## 검증하기

### 오버피팅
- 일반화를 방해 -> 예측 융통성 부족
- 검증 데이터셋 도입하여 해결
    - 검증은 최적화를 진행하지 않음
- 테스트 데이터셋을 추가하여 검증 데이터로의 편향을 막음

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#### 데이터 전처리

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

# 캘리포니아 주택 정보
california = fetch_california_housing()
df = pd.DataFrame(california.data, columns=california.feature_names)
df["Target"] = california.target

# 입출력 데이터 구성
data = torch.from_numpy(df.values).float()
x = data[:, :-1]
y = data[:, -1:]
print(x.size(), y.size())

#### 검증 데이터셋

##### 학습 : 검증 : 평가 = 6 : 2 : 2

In [ ]:
# 학습, 검증, 평가 데이터 비율 선정
ratios = [.6, .2, .2]

# 각 데이터셋에 속하는 데이터 개수 계산
train_cnt = int(data.size(0) * ratios[0])
valid_cnt = int(data.size(0) * ratios[1])
test_cnt = data.size(0) - train_cnt - valid_cnt
cnts = [train_cnt, valid_cnt, test_cnt]

print("Train %d / Valid %d / Test %d samples." % (train_cnt, valid_cnt, test_cnt))

# 입출력 데이터 임의의 순서로 섞기
indices = torch.randperm(data.size(0))
x = torch.index_select(x, dim=0, index=indices)
y = torch.index_select(y, dim=0, index=indices)

# 학습, 검증, 평가 데이터로 나누기
x = list(x.split(cnts, dim=0))
y = y.split(cnts, dim=0)

for x_i, y_i in zip(x, y):
    print(x_i.size(), y_i.size())

# 데이터셋 정규화 하기: 학습 데이터에 대해 피팅된 스케일러(scaler)를 이용하여 남저ㅣ 데이터도 정규화
scaler = StandardScaler()
scaler.fit(x[0].numpy()) # You must fit with train data only.

x[0] = torch.from_numpy(scaler.transform(x[0].numpy())).float()
x[1] = torch.from_numpy(scaler.transform(x[1].numpy())).float()
x[2] = torch.from_numpy(scaler.transform(x[2].numpy())).float()

#### 모델 생성

In [ ]:
# 모델 생성 및 옵티파이저 설정
model = nn.Sequential(
    nn.Linear(x[0].size(-1), 8),
    nn.LeakyReLU(),
    nn.Linear(8, 7),
    nn.LeakyReLU(),
    nn.Linear(7, 6),
    nn.LeakyReLU(),
    nn.Linear(6, 5),
    nn.LeakyReLU(),
    nn.Linear(5, 3),
    nn.LeakyReLU(),
    nn.Linear(3, 3),
    nn.LeakyReLU(),
    nn.Linear(3, y[0].size(-1)),
)

optimizer = optim.Adam(model.parameters())

#### 학습

In [ ]:
# 학습에 필요한 세팅값 설정
n_epochs = 4000
batch_size = 256
print_interval = 100

# 최저 검증 손실 모델 복원을 위한 세팅값 설정
from copy import deepcopy

lowest_loss = np.inf
best_model = None   # 최상의 모델

early_stop = 100    # 학습의 의미가 없으면 일찍 종료
lowest_epoch = np.inf

# 학습 시작
# 훈련, 검증 데이터 손실값 리스트 -> 손실 그래프
train_history, valid_history = [], []

for i in range(n_epochs):
    # 임의의 순서로 미니배치로 나누기
    indices = torch.randperm(x[0].size(0))
    x_ = torch.index_select(x[0], dim=0, index=indices)
    y_ = torch.index_select(y[0], dim=0, index=indices)
    
    x_ = x_.split(batch_size, dim=0)
    y_ = y_.split(batch_size, dim=0)
    
    train_loss, valid_loss = 0, 0
    y_hat = []
    
    # 경사하강법
    for x_i, y_i in zip(x_, y_):
        y_hat_i = model(x_i)
        loss = F.mse_loss(y_hat_i, y_i)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()   

        train_loss += float(loss)

    # 손실값
    train_loss = train_loss / len(x_)

    # 검증시작: 과적합 방지, 미분 수행 x
    with torch.no_grad():
        x_ = x[1].split(batch_size, dim=0)
        y_ = y[1].split(batch_size, dim=0)
        
        valid_loss = 0
        
        for x_i, y_i in zip(x_, y_):
            y_hat_i = model(x_i)
            loss = F.mse_loss(y_hat_i, y_i)
            
            valid_loss += loss
            
            y_hat += [y_hat_i]
            
    valid_loss = valid_loss / len(x_)
    
    train_history += [train_loss]
    valid_history += [valid_loss]
        
    if (i + 1) % print_interval == 0:
        print('Epoch %d: train loss=%.4e  valid_loss=%.4e  lowest_loss=%.4e' % (
            i + 1,
            train_loss,
            valid_loss,
            lowest_loss,
        ))

    # 최소 검증 손실값 갱신 및 모델 저장: 현재 검증 손실값과 비교
    if valid_loss <= lowest_loss:
        lowest_loss = valid_loss
        lowest_epoch = i
        
        # 최소 검증 손실값을 가지는 모델(best model)을 깊은 복사 후 저장, 나중에 모델 불러오기 가능
        # state_dict()는 key-value로 가중치 모델을 반환함.
        best_model = deepcopy(model.state_dict())
    else:   # 가장 낮은 손실값을 가지는 에포크에서 시작해서 early_stop(100회)만큼 실행했는데 
            # 손실값이 오히려 올라가면 학습이 무의미하다.
        if early_stop > 0 and lowest_epoch + early_stop < i + 1:
            print("There is no improvement during last %d epochs." % early_stop)
            break

print("The best validation loss from epoch %d: %.4e" % (lowest_epoch + 1, lowest_loss))

# Load best epoch's model.
model.load_state_dict(best_model)

plot_from = 10

plt.figure(figsize=(20, 10))
plt.grid(True)
plt.title("Train / Valid Loss History")
plt.plot(
    range(plot_from, len(train_history)), train_history[plot_from:],
    range(plot_from, len(valid_history)), valid_history[plot_from:],
)
plt.yscale('log')
plt.show()

# 평가(테스트) 시작
test_loss = 0
y_hat = []

with torch.no_grad():
    x_ = x[2].split(batch_size, dim=0)
    y_ = y[2].split(batch_size, dim=0)

    for x_i, y_i in zip(x_, y_):
        y_hat_i = model(x_i)
        loss = F.mse_loss(y_hat_i, y_i)

        test_loss += loss # Gradient is already detached.

        y_hat += [y_hat_i]

#### 평가

In [ ]:
test_loss = test_loss / len(x_)
y_hat = torch.cat(y_hat, dim=0)

# 훈련, 검증 손실값들을 한데 묶어 정렬
sorted_history = sorted(zip(train_history, valid_history), key=lambda x: x[1])

print("Train loss: %.4e" % sorted_history[0][0])
print("Valid loss: %.4e" % sorted_history[0][1])
print("Test loss: %.4e" % test_loss)

df = pd.DataFrame(torch.cat([y[2], y_hat], dim=1).detach().numpy(),
                  columns=["y", "y_hat"])

sns.pairplot(df, height=5)
plt.show()